In [1]:
import sys
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.results as res
from biogeme.exceptions import biogemeError
from biogeme.expressions import Beta, Variable, bioDraws, MonteCarlo, log, exp
import biogeme.distributions as dist
import matplotlib.pyplot as plt


In [2]:
# Read the data
micro_pool_socio = pd.read_csv('Data/micro_pool_socio_bio2up.csv')

In [3]:
# Micromobility Trips 0-5 Miles
micro0to5 = (((
# 0-2 miles first choice
micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
# 0-2 miles second choice
    # scooter_av == 1 & dlbike_av == 1 & everything else == 0
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
    #'scooter'_av ==1 & 'dk_bike'_av == 1 & everything else == 0
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
    #'dk_bike_av' == 1 & 'dl_bike_av' == 1 & everything else == 0
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['scooter_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0
    )) | ((
# 2-5 miles
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 0) & (
    micro_pool_socio['dkbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 1
)))

micro_0_to_5 = micro_pool_socio.loc[micro0to5]

In [4]:
database4 = db.Database('micro_0_to_5', micro_0_to_5)
globals().update(database4.variables)

In [8]:
ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 1)
ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)

# B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
# B_SCOOTERTIME_S = Beta('B_SCOOTERTIME_S', 1, None, None, 0)
# B_SCOOTERTIME_RND = -exp(B_SCOOTERTIME + B_SCOOTERTIME_S*bioDraws('B_SCOOTERTIME_RND', 'HALTON'))
# B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
# # B_DKBIKETIME = Beta('B_DKBIKETIME', 0, None, None, 0)
# B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_TIME = Beta('B_TIME', 0, None, None, 0)
B_TIME_S = Beta('B_TIME_S', 1, None, None, 0)
B_TIME_RND = -exp(B_TIME + B_TIME_S * bioDraws('B_TIME_RND', 'NORMAL_HALTON2'))

B_COST = Beta('B_COST', 0, None, None, 0)
# B_COST_S = Beta('B_COST_S', 1, None, None, 0)
# B_COST_RND = B_COST + B_COST_S * bioDraws('B_COST_RND', 'NORMAL')

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

# B_costadj = Beta('B_costadj', 0, None, None, 0)

B_PRCP_Yes = Beta('B_PRCP_Yes', 0, None, None, 0)
B_OWNBIKE = Beta('B_OWNBIKE', 0, None, None, 0)
B_BIKELANE = Beta('B_BIKELANE', 0, None, None, 0)
B_AGE_SQ_BIKE = Beta('B_AGE_SQ_BIKE', 0, None, None, 0)
B_AGE_SCTRANSIT_SQ = Beta('B_AGE_SCTRANSIT_SQ', 0, None, None, 0)
B_GENDER_F_BIKE = Beta('B_GENDER_F_BIKE', 0, None, None, 0)
B_GENDER_F_SCTRANSIT = Beta('B_GENDER_F_SCTRANSIT', 0, None, None, 0)

In [9]:
# Utility Functions
V7 = (ASC_SCOOTER 
      + B_TIME_RND*sctime
      + B_COST*sccost_adj
      + B_ACCESS*SCAW*(1-AVtech)
      + B_WAITAV*SCAV*AVtech 
      )
V8 = (ASC_DLBIKE
        + B_TIME_RND*dltime
        + B_COST*dlcost_adj
        + B_ACCESS*DLAW
       # + B_PRCP_Yes*PRCP_Yes
        + B_OWNBIKE*bike
        # + B_BIKELANE*BKLN_Yes
        + B_AGE_SQ_BIKE*age*age
        + B_GENDER_F_BIKE*gender_F
    )
V9 = (ASC_DKBIKE 
      + B_TIME_RND*dbtime 
      + B_COST*dbcost_adj
      + B_ACCESS*DBAW
      + B_DROP*DBDW
      + B_OWNBIKE*bike
      # + B_BIKELANE*BKLN_Yes
      + B_AGE_SQ_BIKE*age*age
      + B_GENDER_F_BIKE*gender_F
    )
V10 = (ASC_SCTRANSIT
       + B_TIME_RND*sttotime
        + B_COST*sttocost_adj 
        + B_ACCESS*STAW*(1-AVtech)
        + B_WAITAV*STAV*AVtech 
        + B_AGE_SCTRANSIT_SQ*age*age
        + B_GENDER_F_SCTRANSIT*gender_F
    )

In [10]:
V = {7: V7, 8: V8, 9: V9, 10: V10}
av = {7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av}

prob = models.logit(V, av, choice)
logprob = log(MonteCarlo(prob))
biogeme = bio.BIOGEME(database4, logprob, numberOfDraws = 100)
biogeme.modelName = "micromobility_mixedlogit_0_to_5"

results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults.round(2))

                       Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_DKBIKE             -0.43          0.17        -2.57          0.01
ASC_DLBIKE              0.21          0.15         1.37          0.17
ASC_SCTRANSIT          -0.14          0.13        -1.05          0.29
B_ACCESS               -0.12          0.02        -8.17          0.00
B_AGE_SCTRANSIT_SQ      0.00          0.00         0.27          0.79
B_AGE_SQ_BIKE           0.00          0.00         1.58          0.11
B_COST                 -1.89          0.32        -5.92          0.00
B_DROP                 -0.08          0.03        -2.63          0.01
B_GENDER_F_BIKE         0.08          0.11         0.71          0.48
B_GENDER_F_SCOTRANSIT  -0.15          0.12        -1.25          0.21
B_OWNBIKE              -0.18          0.13        -1.45          0.15
B_TIME                 -3.82          0.77        -5.00          0.00
B_TIME_S                2.00          0.81         2.46          0.01
B_WAITAV            

In [25]:
print(logLike)

-2650.4262493268243
